In [1]:
import os
from pathlib import Path

import numpy as np
import cv2
import supervision as sv
from autodistill.detection import CaptionOntology
from autodistill_grounded_sam import GroundedSAM
# from autodistill_yolov8 import YOLOv8
from ultralytics import YOLO
from IPython.display import Image
import matplotlib.pyplot as plt
from boxmot import (OCSORT, BoTSORT, BYTETracker, DeepOCSORT, StrongSORT,
                    create_tracker, get_tracker_config)

import video_multiprocessing as vm
import superimpose_boxes as sb

HOME = os.getcwd()
HOME

CUDA not available. Using MPS.
CUDA not available. Using MPS.


'/Users/eric/Desktop/2-Career/Projects/ObjectDetection'

In [2]:
# project = 'license_plates' # 'dog_park' # 'hockey'
project = 'dog_park' # 'dog_park' # 'hockey'
# project = 'hockey' # 'dog_park' # 'hockey'
if project == 'license_plates':
    # latest_train_dir = input("Enter the name of the latest training directory: ")
    latest_train_dir = 'train_LP_2023.12.26_14.22.25_n_im640_ep20_baNone_seNone'

    # LP: 6, 10, 14, 32
    source = "license_plates/videos/6.mp4"
    # source = "license_plates/videos/10.mp4"
    # source = "license_plates/videos/14.mp4"
    # source = "license_plates/videos/32.mp4"
elif project == 'dog_park':
    # latest_train_dir = 'train_dogpark_2023.12.26_17.08.07_n_im640_ep100_baNone_seNone'
    latest_train_dir = 'train_dog_park_2023.12.27_11.39.50_n_im1088_ep100_baNone_seNone'
    source = "dog_park/videos/dogpark_trim_8m.mp4"
    # source = "dog_park/videos/dog6s.mp4"
    source = "dog_park/videos/glacier_night.mp4"
elif project == 'hockey':
    latest_train_dir = 'train_hockey_2023.12.26_17.08.07_n_im640_ep100_baNone_seNone'
    source = "hockey/videos/003.mp4"
    # source = "hockey/videos/003.mp4"
    # source = "hockey/videos/002.mp4"
    # source = "hockey/videos/002_trim.mp4"



weights = 'best' 
# weights = 'last' 
model = YOLO(f"runs/detect/{latest_train_dir}/weights/{weights}.pt")
# model = YOLO(f"yolov8x.pt")
!tree {HOME}/runs/detect/{latest_train_dir}/

model.predict(source, save=True)
# TODO: I think using MPS for Inference causes an issue with the bounding boxes. Find out why.

/Users/eric/Desktop/2-Career/Projects/ObjectDetection/runs/detect/train_dog_park_2023.12.27_11.39.50_n_im1088_ep100_baNone_seNone/
├── F1_curve.png
├── PR_curve.png
├── P_curve.png
├── R_curve.png
├── args.yaml
├── confusion_matrix.png
├── events.out.tfevents.1703698791.Erics-Personal-MacBook-Pro-2023.local.20053.0
├── labels.jpg
├── labels_correlogram.jpg
├── results.csv
├── results.png
├── train_batch0.jpg
├── train_batch1.jpg
├── train_batch2.jpg
├── val_batch0_labels.jpg
├── val_batch0_pred.jpg
├── val_batch1_labels.jpg
├── val_batch1_pred.jpg
├── val_batch2_labels.jpg
├── val_batch2_pred.jpg
└── weights
    ├── best.pt
    └── last.pt

2 directories, 22 files




    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/3385) /Users/eric/Desktop/2-Career/Projects/ObjectDetection/dog_park/videos/glacier_night.mp4: 640x1088 (no detections), 286.8ms
video 1/1 (2/3385) /Users/eric/Desktop/2-Career/Projects/ObjectDetection/dog_park/videos/glacier_night.mp4: 640x1088 (no detections), 264.9ms
video 1/1 (3/3385) /Users/eric/Desktop/2-Career/Projects/ObjectDetection/dog_park/videos/glacier_night.mp4: 640x1088 (no detections), 153.9ms
video 1/1 (4/3385) /Users/eric/De

[ultralytics.yolo.engine.results.Results object with attributes:
 
 boxes: ultralytics.yolo.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'dog'}
 orig_img: array([[[ 39,  29,  33],
         [ 39,  29,  33],
         [ 39,  29,  33],
         ...,
         [ 68,  85,  93],
         [ 68,  85,  93],
         [ 68,  85,  93]],
 
        [[ 38,  28,  32],
         [ 38,  28,  32],
         [ 38,  28,  32],
         ...,
         [ 68,  85,  93],
         [ 68,  85,  93],
         [ 68,  85,  93]],
 
        [[ 38,  28,  32],
         [ 38,  28,  32],
         [ 38,  28,  32],
         ...,
         [ 68,  85,  93],
         [ 68,  85,  93],
         [ 68,  85,  93]],
 
        ...,
 
        [[ 96, 120, 129],
         [ 96, 120, 129],
         [ 93, 117, 126],
         ...,
         [ 68,  78,  83],
         [ 68,  78,  83],
         [ 68,  78,  83]],
 
        [[ 96, 120, 129],
         [ 95, 119, 128],
         [ 93, 117, 126],
         ...,
     

In [16]:
def track(sources):
    def create_video_writer(video_cap, output_filename):
        frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(video_cap.get(cv2.CAP_PROP_FPS))
        fourcc = cv2.VideoWriter_fourcc(*'avc1')
        writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))
        return writer

    if isinstance(sources, str):
        sources = [sources]

    for source in sources:
        class_dict = {0: ('license plate', (0, 10, 255))}  # red color for license plate
        tracker = DeepOCSORT(model_weights=Path('osnet_x0_25_msmt17.pt'), device="mps:0", fp16=True)

        vid = cv2.VideoCapture(source)
        output_filename = f"/Users/eric/Desktop/2-Career/Projects/ObjectDetection/license_plates/tracked_videos/{source.split('/')[-1].split('.')[0]}_DeepOCSORT.mp4"
        writer = create_video_writer(vid, output_filename)

        while True:
            ret, image = vid.read()
            if not ret:
                break

            detections = model.predict(image)[0]
            det_array = np.array(detections.boxes.data.tolist())

            if det_array.size != 0:
                if det_array.ndim == 1:
                    det_array = np.expand_dims(det_array, axis=0)
                ts = tracker.update(det_array, image)

                if ts.ndim == 2 and ts.size > 0:
                    xyxys = ts[:, 0:4].astype('int')  # Convert to int
                    ids = ts[:, 4].astype('int')
                    confs = ts[:, 5]
                    clss = ts[:, 6].astype('int')  # Assuming class information is in the 7th column (index 6)

                    for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
                        image = cv2.rectangle(image, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), class_dict[cls][1], 1)
                        cv2.putText(image, f"{class_dict[cls][0]} {int(round(conf, 2) * 100 )}% id:{id}", (xyxy[0], xyxy[1]-10), cv2.FONT_HERSHEY_DUPLEX, 0.6, class_dict[cls][1], 2)

            writer.write(image)

        vid.release()
        writer.release()

source = "license_plates/videos/32.mp4"
latest_train_dir = 'train_2023.12.26_14.22.25_n_im640_ep20_baNone_seNone'
# latest_train_dir = input("Enter the name of the latest training directory: ")
weights = 'best' 
# # weights = 'last' 
model = YOLO(f"runs/detect/{latest_train_dir}/weights/{weights}.pt")
track(source)

2023-12-26 16:55:48.885 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-12-26 16:55:48.887 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')

0: 384x640 1 license plate, 487.8ms
Speed: 1.3ms preprocess, 487.8ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 license plate, 497.9ms
Speed: 2.1ms preprocess, 497.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 license plate, 956.2ms
Speed: 3.5ms preprocess, 956.2ms inference, 7.6ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 license plate, 981.3ms
Speed: 9.2ms preprocess, 981.3ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 license plate, 840.8ms
Speed: 0.4ms preprocess, 840.8ms i

In [4]:
'''This worked for hockey but not for license plates - maybe because there was never a frame with 0 detections in hockey'''
# def track(sources):

#     def create_video_writer(video_cap, output_filename):

#         # grab the width, height, and fps of the frames in the video stream.
#         frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#         frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#         fps = int(video_cap.get(cv2.CAP_PROP_FPS))

#         # initialize the FourCC and a video writer object
#         fourcc = cv2.VideoWriter_fourcc(*'avc1') #avc1 instead of mp4v so I can play the video in VSCode
#         writer = cv2.VideoWriter(output_filename, fourcc, fps,
#                                  (frame_width, frame_height))

#         return writer
    
#     if isinstance(sources, str):
#         sources = [sources]

#     for source in sources:
#         class_dict = {
#             # class, color (BGR)
#             0: ('license plate', (0, 10, 255)),  # red
#         }
#         # class_dict = {
#         #     # class, color (BGR)
#         #     0: ('Skater', (0, 10, 255)),  # red
#         #     1: ('Goalie', (0, 255, 30)),  # green
#         #     2: ('Referee', (255, 160, 0)),  # blue
#         # }
#         tracker = DeepOCSORT(
#             # which ReID model to use
#             model_weights=Path('osnet_x0_25_msmt17.pt'),
#             device="mps:0",
#             fp16=True,  # True if gpu available
#         )

#         vid = cv2.VideoCapture(source)
#         writer = create_video_writer(
#             vid, f"/Users/eric/Desktop/2-Career/Projects/ObjectDetection/license_plates/tracked_videos/{source.split('/')[-1].split('.')[0]}_DeepOCSORT.mp4")

#         while True:
#             ret, image = vid.read()
#             detections = model.predict(image)[0]

#             # initialize the list of bounding boxes and confidences
#             results = []
#             if not ret:
#                 break

#             if np.array(detections.boxes.data.tolist()).ndim < 2:
#                 # (x, y, x, y, id, conf, cls)
#                 results = [[0, 0, 0, 0, 0.0922948837280273, 0]]
                
#             ts = tracker.update(np.array(detections.boxes.data.tolist()), image)

#             xyxys = ts[:, 0:4].astype('int')  # float64 to int
#             ids = ts[:, 4].astype('int')  # float64 to int
#             confs = ts[:, 5]
#             clss = ts[:, 6]
#             if ts.shape[0] != 0:
#                 for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
#                     image = cv2.rectangle(
#                         image,
#                         (xyxy[0], xyxy[1]),
#                         (xyxy[2], xyxy[3]),
#                         class_dict[cls][1],
#                         1
#                     )
#                     cv2.putText(
#                         img=image,
#                         text=f"{class_dict[cls][0]} {f'{int(round(conf, 2) * 100 )}%'} id:{id}",
#                         org=(xyxy[0], xyxy[1]-10),
#                         fontFace=cv2.FONT_HERSHEY_DUPLEX,
#                         fontScale=.6,
#                         color=class_dict[cls][1],
#                         thickness=2
#                     )
#             writer.write(image)
#         vid.release()
#         writer.release()

# # latest_train_dir = input("Enter the name of the latest training directory: ")
# latest_train_dir = 'train_2023.12.26_14.22.25_n_im640_ep20_baNone_seNone'
# weights = 'best' 
# # # weights = 'last' 
# model = YOLO(f"runs/detect/{latest_train_dir}/weights/{weights}.pt")
# source = "license_plates/videos/10.mp4"
# track(source)

'This worked for hockey but not for license plates - maybe because there was never a frame with 0 detections in hockey'